In [1]:
import pandas as pd
import numpy as np

In [3]:
df_matches = pd.read_csv('IPL Matches 2008-2020.csv')
df_matches.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [4]:
df_deliveries = pd.read_csv('IPL Ball-by-Ball 2008-2020.csv')
df_deliveries.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [5]:
df_total = pd.merge(df_matches, df_deliveries, left_on='id', right_on='id',how='outer')
df_total = df_total.drop(['umpire1','umpire2'], axis = 1) 
df_total.head(2)

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,...,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [6]:
batsman_runs = df_total.groupby(['id','batsman'])['batsman_runs'].sum()
df_batsman_runs = batsman_runs.to_frame().reset_index()
df_batsman_runs.head(2)

,id,batsman,batsman_runs
0,335982,AA Noffke,9
1,335982,B Akhil,0


In [14]:
total_runs = df_total.groupby(['id'])['batsman_runs'].sum()
df_total_runs = total_runs.to_frame().reset_index()
df_total_runs.rename(columns = {'batsman_runs' : 'total_score'}, inplace = True)
df_total_runs.head(2)
#Adding runs by grouping ids. Needs verification

,id,total_score
0,335982,268
1,335983,430


In [15]:
dfballs = df_total[(df_total['extra_runs'] == 0) ] 
batsman_balls_faced = dfballs.groupby(['id','batsman'])['batsman_runs'].count() 
df_batsman_balls_faced = batsman_balls_faced.to_frame().reset_index()
df_batsman_balls_faced.rename(columns = {'batsman_runs' : 'balls_faced'}, inplace = True)
df_batsman_balls_faced.head(2) 

,id,batsman,balls_faced
0,335982,AA Noffke,10
1,335982,B Akhil,2


In [16]:
df_strike_rate = df_batsman_runs.merge(df_batsman_balls_faced, how = 'inner', on = ['id', 'batsman'])
df_strike_rate['strike_rate'] = round(((df_strike_rate['batsman_runs'] / df_strike_rate['balls_faced']) * 100),2)
df_strike_rate.head(2)

,id,batsman,batsman_runs,balls_faced,strike_rate
0,335982,AA Noffke,9,10,90.0
1,335982,B Akhil,0,2,0.0


In [17]:
df_strike_rate = df_strike_rate.merge(df_total_runs, how = 'outer', on = ['id'])

In [18]:
df_strike_rate.head(2)

,id,batsman,batsman_runs,balls_faced,strike_rate,total_score
0,335982,AA Noffke,9,10,90.0,268
1,335982,B Akhil,0,2,0.0,268


In [19]:
df_strike_rate.head()

,id,batsman,batsman_runs,balls_faced,strike_rate,total_score
0,335982,AA Noffke,9,10,90.00,268
1,335982,B Akhil,0,2,0.00,268
2,335982,BB McCullum,158,71,222.54,268
3,335982,CL White,6,10,60.00,268
4,335982,DJ Hussey,12,12,100.00,268


In [22]:
df_battingteam= df_total[['id','batsman','batting_team']]
df_team = df_strike_rate.merge(df_battingteam, how = 'inner', on = ['id', 'batsman'])
df_team= df_team.drop_duplicates(subset=['id','batsman','balls_faced','strike_rate'])
df_team.rename(columns = {'batting_team' : 'Team'}, inplace = True)
df_team.head(3)

,id,batsman,batsman_runs,balls_faced,strike_rate,total_score,Team
0,335982,AA Noffke,9,10,90.00,268,Royal Challengers Bangalore
12,335982,B Akhil,0,2,0.00,268,Royal Challengers Bangalore
14,335982,BB McCullum,158,71,222.54,268,Kolkata Knight Riders


In [23]:
df_bowlingteam = df_total[['id','batsman','bowling_team']]
df_opposition = df_team.merge(df_bowlingteam, how = 'inner', on = ['id', 'batsman'])
df_opposition= df_opposition.drop_duplicates(subset=['id','batsman','batsman_runs','balls_faced','strike_rate','Team'])
df_opposition.rename(columns = {'bowling_team' : 'opposition_team'}, inplace = True)
df_opposition.head(3)

,id,batsman,batsman_runs,balls_faced,strike_rate,total_score,Team,opposition_team
0,335982,AA Noffke,9,10,90.00,268,Royal Challengers Bangalore,Kolkata Knight Riders
12,335982,B Akhil,0,2,0.00,268,Royal Challengers Bangalore,Kolkata Knight Riders
14,335982,BB McCullum,158,71,222.54,268,Kolkata Knight Riders,Royal Challengers Bangalore


In [24]:
df_overs = df_total[(df_total['extra_runs'] == 0)]

bowlerovers = df_overs.groupby(['id','bowler'])['ball'].count()/6
df_bowlersovers = bowlerovers.to_frame().reset_index()
df_bowlersovers["ball"] = df_bowlersovers['ball'].astype(int)
df_bowlersovers.rename(columns={'ball':'overs'}, inplace=True)
df_bowlersovers.head()

,id,bowler,overs
0,335982,AA Noffke,3
1,335982,AB Agarkar,4
2,335982,AB Dinda,3
3,335982,CL White,1
4,335982,I Sharma,2


In [25]:
df_runs = df_total.groupby(['id','bowler'],as_index=False)['batsman_runs','extra_runs'].sum()
df_runs['runs_conceded'] = df_runs['batsman_runs'] + df_runs['extra_runs']  
df_runs = df_runs[['id','bowler','runs_conceded']]
df_runs.head()
df_runs = pd.merge(df_bowlersovers, df_runs, left_on=['id','bowler'], right_on=['id','bowler'],how='outer')
df_runs.head()

,id,bowler,overs,runs_conceded
0,335982,AA Noffke,3.0,41
1,335982,AB Agarkar,4.0,25
2,335982,AB Dinda,3.0,9
3,335982,CL White,1.0,24
4,335982,I Sharma,2.0,13


In [26]:
df_wickets = df_total[(df_total['player_dismissed'] != 'No Wicket') & (df_total['dismissal_kind'] != 'run out')]

df_wickets_taken = df_wickets.groupby(['id','bowler'],as_index=False)['player_dismissed'].count()
df_wickets_taken.rename(columns={'player_dismissed':'wickets_taken'},inplace=True)
df_wickets_taken.head()
df_wickets_taken = pd.merge(df_runs, df_wickets_taken, left_on=['id','bowler'], right_on=['id','bowler'],how='outer')
df_wickets_taken.head()

,id,bowler,overs,runs_conceded,wickets_taken
0,335982,AA Noffke,3.0,41,1.0
1,335982,AB Agarkar,4.0,25,3.0
2,335982,AB Dinda,3.0,9,2.0
3,335982,CL White,1.0,24,0.0
4,335982,I Sharma,2.0,13,1.0


In [27]:
df_balls = df_total[(df_total['extra_runs'] == 0) ]

df_ballsbowled = df_balls.groupby(['id','bowler'],as_index=False)['ball'].count()
df_economy = pd.merge(df_runs, df_ballsbowled, left_on=['id','bowler'], right_on=['id','bowler'],how='outer')
df_economy.head()
df_economy['economy_rate'] = round((df_economy['runs_conceded'] / df_economy['ball'])*6,2)
df_economy = df_economy[['id','bowler','economy_rate']]
df_economy.head()
df_economy = pd.merge(df_wickets_taken, df_economy, left_on=['id','bowler'], right_on=['id','bowler'],how='outer')
df_economy.head()

,id,bowler,overs,runs_conceded,wickets_taken,economy_rate
0,335982,AA Noffke,3.0,41,1.0,10.70
1,335982,AB Agarkar,4.0,25,3.0,6.25
2,335982,AB Dinda,3.0,9,2.0,3.00
3,335982,CL White,1.0,24,0.0,24.00
4,335982,I Sharma,2.0,13,1.0,5.20


In [28]:
df_final = pd.merge(df_opposition, df_economy, left_on=['id'], right_on=['id'],how='outer')

In [30]:
df_final.head()

,id,batsman,batsman_runs,balls_faced,strike_rate,total_score,Team,opposition_team,bowler,overs,runs_conceded,wickets_taken,economy_rate
0,335982,AA Noffke,9,10,90.0,268,Royal Challengers Bangalore,Kolkata Knight Riders,AA Noffke,3.0,41,1.0,10.70
1,335982,AA Noffke,9,10,90.0,268,Royal Challengers Bangalore,Kolkata Knight Riders,AB Agarkar,4.0,25,3.0,6.25
2,335982,AA Noffke,9,10,90.0,268,Royal Challengers Bangalore,Kolkata Knight Riders,AB Dinda,3.0,9,2.0,3.00
3,335982,AA Noffke,9,10,90.0,268,Royal Challengers Bangalore,Kolkata Knight Riders,CL White,1.0,24,0.0,24.00
4,335982,AA Noffke,9,10,90.0,268,Royal Challengers Bangalore,Kolkata Knight Riders,I Sharma,2.0,13,1.0,5.20


In [31]:
df_final.to_csv('final.csv')